In [1]:
! pip install pixivpy

  Using cached https://files.pythonhosted.org/packages/c7/6f/3054e4e2a544095de204d4f2bb60bb7d640461f570e1af2f23a469d328cc/PixivPy-3.5.11-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/c1/24814557f1d22c56d50280771a17307e6bf87b70727d975fd6b2ce6b014a/requests-2.25.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/ef/7681134338fc097acef8d9b2f8abe0458e4d87559c689a8c306d0957ece5/requests_toolbelt-0.9.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/83/e4/f1d3872ce822f52f4133cc04960185f64676ca5ec87f05a5350fc0c0a92f/cloudscraper-1.2.48-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/c7/fa589626997dd07bd87d9269342ccb74b1720384a4d739a1872bd84fbe68/chardet-4.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/a0/5f06e1e1d463903cf0c0eebeb751791119ed7a4b3737fdc9a77f1cdfb51f/certifi-2020.12.5-py2.py3-none-any.whl
    100% |█████████████████████████████

In [1]:
def get_illusts(date):
    illusts = []
    json_result = api.illust_ranking('day', date=date)
    if json_result is None: return illusts
    if json_result.illusts is None: return illusts
    illusts.extend(json_result.illusts)
    while True:
        if json_result.next_url is None: return illusts
        next_qs = api.parse_qs(json_result.next_url)
        if next_qs is None: return illusts
        json_result = api.illust_ranking(**next_qs)
        if json_result is None: return illusts
        if json_result.illusts is None: return illusts
        illusts.extend(json_result.illusts)

In [2]:
from pixivpy3 import *
from datetime import date, timedelta

refresh_token = 'GwWqT83JBSi4M_mBqs6B6za5cH9xeq0gRlI69zYlfvk' 

api = AppPixivAPI()
api.auth(refresh_token=refresh_token) # api.login("username", "password")   # Not required

{'access_token': '_R2WwHdl5oVDWvaPI0bK8aUaziNIyUZun30r-hORDPo',
 'expires_in': 3600,
 'token_type': 'bearer',
 'scope': '',
 'refresh_token': 'GwWqT83JBSi4M_mBqs6B6za5cH9xeq0gRlI69zYlfvk',
 'user': {'profile_image_urls': {'px_16x16': 'https://s.pximg.net/common/images/no_profile_ss.png',
   'px_50x50': 'https://s.pximg.net/common/images/no_profile_s.png',
   'px_170x170': 'https://s.pximg.net/common/images/no_profile.png'},
  'id': '64763103',
  'name': 'eyesssewnshut',
  'account': 'user_chrm5557',
  'mail_address': 'followdemiurg@gmail.com',
  'is_premium': False,
  'x_restrict': 1,
  'is_mail_authorized': True},
 'response': {'access_token': '_R2WwHdl5oVDWvaPI0bK8aUaziNIyUZun30r-hORDPo',
  'expires_in': 3600,
  'token_type': 'bearer',
  'scope': '',
  'refresh_token': 'GwWqT83JBSi4M_mBqs6B6za5cH9xeq0gRlI69zYlfvk',
  'user': {'profile_image_urls': {'px_16x16': 'https://s.pximg.net/common/images/no_profile_ss.png',
    'px_50x50': 'https://s.pximg.net/common/images/no_profile_s.png',


In [3]:
good_images = [] 

for i in range(150):
    curr_date = str(date.today() - timedelta(3 * i  + 1))
    illusts   = get_illusts(curr_date)
    for illust in illusts:
        if illust.type == 'manga': continue
        good_images.append(illust.id)

In [4]:
good_images = set(good_images)

In [5]:
import pickle

with open('good_images.pickle', 'wb') as handle:
    pickle.dump(good_images, handle, pickle.HIGHEST_PROTOCOL)

In [14]:
import re

def preprocess_tag(tag):
    
    tag = tag.lower()
    tag = re.sub(r'\s\d+\+\s(bookmarks)', ' ', tag)
    tag = tag.strip()
    
    # TODO: add more preprocessing?
    
    return tag

In [17]:
with open('good_images.pickle', 'rb') as handle: good_img = pickle.load(handle)

In [26]:
popular_tags = []

for i in range(150):
    curr_date = str(date.today() - timedelta(3 * i  + 1))
    illusts   = get_illusts(curr_date)
    for illust in illusts:
        
        if illust.type == 'manga': continue
        if not illust.id in good_img: continue
            
        tags = [preprocess_tag(t['translated_name']) 
                for t in illust.tags if t['translated_name'] is not None]
        tags = list(set(tags))
        
        popular_tags.extend(tags)
        good_img.remove(illust.id)
        

In [27]:
from collections import Counter

popular_tags = dict(Counter(popular_tags))
popular_tags = {k: v for k, v in popular_tags.items() if v >= 20}
popular_tags = {k: v for k, v in sorted(popular_tags.items(), key = lambda x: x[1], reverse=True)}

In [28]:
with open('popular_tags.pickle', 'wb') as handle:
    pickle.dump(popular_tags, handle, pickle.HIGHEST_PROTOCOL)

In [25]:
popular_tags

{'original': 86,
 'genshin impact': 38,
 'girl': 81,
 'horse girl': 29,
 'uma musume pretty derby': 45}